In [ ]:
def generate_func(sample, distro, *args, n_samples=200):
    if distro == 'gaussian' or distro == 'normal':
        def var_func(var):
            cdf = scipy.stats.norm(scale=var).cdf
            return compute_ksstat(sample, cdf)
        return var_func
    elif distro == 'laplace':
        def var_func(var):
            cdf = scipy.stats.laplace(scale=var).cdf
            return compute_ksstat(sample, cdf)
        return var_func
    elif distro == 't':
        def var_func(var):
            cdf = scipy.stats.t(df=2, scale=var).cdf
            return compute_ksstat(sample, cdf)
        return var_func
    elif distro == 'prior_r':
        eta = args[0]
        def r_func(r):
            cdf = compute_prior_cdf(r, eta, n_samples=n_samples)
            return compute_ksstat(sample, cdf)
        return r_func
    elif distro == 'prior_eta':
        r = args[0]
        def eta_func(eta):
            cdf = compute_prior_cdf(r, eta, n_samples=n_samples)
            return compute_ksstat(sample, cdf)
        return eta_func
    elif distro == 'prior':
        def r_eta_func(params):
            r = params[0]
            eta = params[1]
            cdf = compute_prior_cdf(r, eta, n_samples=n_samples, debug=False)
            return compute_ksstat(sample, cdf)
        return r_eta_func

    print("Please enter a valid argument for `distro`: 'gaussian', 'laplace', 'prior_r', 'prior_eta', 'prior', 't'")

for i, group in enumerate(GROUPS):
    initial_param = np.array([master_df.loc[group, 'initial_r'], master_df.loc[group, 'initial_eta']])
    print(f"\n{GROUP.capitalize()} {group}\nInitial param:", initial_param)
    bounds = np.array(list(zip(initial_param - 0.1, initial_param + 0.1))) 
    bounds = np.where(bounds < 0, 0, bounds)
    print("Bounds:", bounds)
    result = scipy.optimize.minimize(generate_func(group_data_map[group], distro='prior'),
                                        x0=initial_param, 
                                        bounds=bounds)
    print(result)
    best_r, best_eta = result.x[0], result.x[1]
    if result.fun < master_df.loc[group, 'kstest_stat_initial']:
        master_df.loc[group, 'best_r'], master_df.loc[group, 'best_eta'] = best_r, best_eta
        master_df.loc[group, 'kstest_stat_best'] = result.fun
    else:
        master_df.loc[group, 'best_r'], master_df.loc[group, 'best_eta'] = master_df.loc[group, 'initial_r'], master_df.loc[group, 'initial_eta'] 
        master_df.loc[group, 'kstest_stat_best'] = master_df.loc[group, 'kstest_stat_initial']
    print(f"% Reduction: {(master_df.loc[group,'kstest_stat_initial'] - result.fun)*100/result.fun}%")
master_df.to_csv(os.path.join(CWD, "CSVs", "master_df_scipyoptimize.csv"))

master_df = pd.read_csv(os.path.join(CWD, "CSVs", "master_df_scipyoptimize.csv"), index_col=GROUP)
master_df